In [3]:
import pandas as pd
import json

## Airport Location Data

#### Filter Data

In [47]:
df_airports = pd.read_csv('./data/all_airport_data.csv')
df_itenary = pd.read_csv('./data/origin_arrival.csv')

# keep only unique id 
df_airports = df_airports.drop_duplicates(subset=['AIRPORT_ID'])

# drop if country_code is not US
df_airports = df_airports[df_airports['AIRPORT_COUNTRY_CODE_ISO'] == 'US']

flight_per_airport = df_itenary.groupby(['ORIGIN_AIRPORT_ID']).size().reset_index(name='FLIGHT_COUNT').sort_values(by='FLIGHT_COUNT', ascending=False)

top_100_us_airports = flight_per_airport.head(100).drop(columns=['FLIGHT_COUNT']).merge(df_airports, left_on='ORIGIN_AIRPORT_ID', right_on='AIRPORT_ID', how='left').drop(columns=['ORIGIN_AIRPORT_ID'])

top_100_us_airports.to_json('./data/airports.json', orient='records')

## Itenary Data
https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FHK&QO_fu146_anzr=b4vtv0%20n0q%20Qr56v0n6v10%20f748rB

### Most Popular routes 

In [30]:
df_itenary = pd.read_csv('./data/origin_arrival.csv')
df_airports = pd.read_json('./data/airports.json')
df_airports = df_airports[['AIRPORT_ID', 'LATITUDE', 'LONGITUDE']]

valid_airport_ids = set(df_airports['AIRPORT_ID'])

print(df_itenary.count())


df_itenary = df_itenary[
    df_itenary['ORIGIN_AIRPORT_ID'].isin(valid_airport_ids) &
    df_itenary['DEST_AIRPORT_ID'].isin(valid_airport_ids)
]

print(df_itenary.count())

flight_counts = df_itenary.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).size().reset_index(name='FLIGHT_COUNT')
flight_counts = flight_counts.sort_values(by=['ORIGIN_AIRPORT_ID', 'FLIGHT_COUNT'], ascending=[True, False])
top_5_destinations_foreach_origin = flight_counts.groupby('ORIGIN_AIRPORT_ID').apply(lambda x: x.nlargest(5, 'FLIGHT_COUNT'))
top_5_destinations_foreach_origin = top_5_destinations_foreach_origin.reset_index(drop=True)
top_5_destinations_foreach_origin.head(100)



ORIGIN_AIRPORT_ID     6714113
DEST_AIRPORT_ID       6714113
PASSENGERS            6714113
MARKET_FARE           6714113
MARKET_DISTANCE       6714113
MARKET_MILES_FLOWN    6714113
dtype: int64
ORIGIN_AIRPORT_ID     4912280
DEST_AIRPORT_ID       4912280
PASSENGERS            4912280
MARKET_FARE           4912280
MARKET_DISTANCE       4912280
MARKET_MILES_FLOWN    4912280
dtype: int64


,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,FLIGHT_COUNT
0,10140,12889,1296
1,10140,14107,1284
2,10140,12892,1237
3,10140,11292,818
4,10140,14747,795
...,...,...,...
95,11278,10721,3260
96,11278,13204,3078
97,11278,12892,3012
98,11278,13930,2846


In [29]:
connections_df = top_5_destinations_foreach_origin.merge(
    df_airports,
    how='left',
    left_on='ORIGIN_AIRPORT_ID',
    right_on='AIRPORT_ID'
).rename(columns={'LATITUDE': 'ORIGIN_AIRPORT_LAT', 'LONGITUDE': 'ORIGIN_AIRPORT_LONGITUDE'})

connections_df = connections_df.drop(columns='AIRPORT_ID')

connections_df = connections_df.merge(
    df_airports,
    how='left',
    left_on='DEST_AIRPORT_ID',
    right_on='AIRPORT_ID'
).rename(columns={'LATITUDE': 'DEST_AIRPORT_LAT', 'LONGITUDE': 'DEST_AIRPORT_LONGITUDE'})


connections_df = connections_df.drop(columns='AIRPORT_ID')


connections_df.to_json('./data/connections.json', orient='records')




,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,FLIGHT_COUNT,ORIGIN_AIRPORT_LAT,ORIGIN_AIRPORT_LONGITUDE,DEST_AIRPORT_LAT,DEST_AIRPORT_LONGITUDE
0,10140,12889,1296,35.041667,-106.606389,36.080000,-115.152222
1,10140,14107,1284,35.041667,-106.606389,33.436111,-112.009444
2,10140,12892,1237,35.041667,-106.606389,33.942500,-118.408056
3,10140,11292,818,35.041667,-106.606389,39.774444,-104.879722
4,10140,14747,795,35.041667,-106.606389,47.447222,-122.305556
5,10257,13204,1818,42.745833,-73.805278,28.431667,-81.324722
6,10257,11697,1178,42.745833,-73.805278,26.074167,-80.151667
7,10257,15304,856,42.745833,-73.805278,27.973056,-82.535278
8,10257,10397,520,42.745833,-73.805278,33.640833,-84.427222
9,10257,11292,518,42.745833,-73.805278,39.774444,-104.879722


### Flight delays

In [4]:
df = pd.read_csv('./data/flight_delays_data.csv')

# Define the bins and labels
bins = [0, 5, 15, 30, 45, 90, float('inf')]
labels = ['<5', '<15', '<30', '<45', '<90', '90+']

# Bin the data
df['DEP_DELAY_BINNED'] = pd.cut(df['DEP_DELAY'], bins=bins, labels=labels, right=False)

# Group by the airport ID and count the number of entries for each bin
grouped = df.groupby('ORIGIN_AIRPORT_ID')['DEP_DELAY_BINNED'].value_counts().unstack(fill_value=0)

# Prepare the data for JSON
result = {}
for airport_id, counts in grouped.iterrows():
    result[airport_id] = {
        'labels': labels,
        'data': (counts * 12).tolist()
    }


# Save the result as a JSON file
with open('./data/delayed_flights.json', 'w') as f:
    json.dump(result, f)

In [21]:
### Airport Scores on Delays, Diversion, Size, Cancellation

In [28]:
df = pd.read_csv('./data/flight_delays_data.csv')

cancelled_counts = df.groupby('ORIGIN_AIRPORT_ID')['CANCELLED'].sum()
delayed_counts = df.groupby('ORIGIN_AIRPORT_ID')['DEP_DEL15'].sum()
diverted_counts = df.groupby('ORIGIN_AIRPORT_ID')['DIVERTED'].sum()
entry_counts = df.groupby('ORIGIN_AIRPORT_ID').size()

cancelled_rate = cancelled_counts / entry_counts
delayed_rate = delayed_counts / entry_counts
diverted_rate = diverted_counts / entry_counts

def normalize(series):
    return 100 * (series - series.min()) / (series.max() - series.min())

normalized_cancelled = normalize(cancelled_rate)
normalized_delayed = normalize(delayed_rate)
normalized_diverted = normalize(diverted_rate)
normalized_entries = normalize(entry_counts)

# Create a DataFrame with the normalized 
scores_df = pd.DataFrame({
    'CANCELLED': normalized_cancelled,
    'DELAYED': normalized_delayed,
    'DIVERTED': normalized_diverted,
    'AIRPORT_SIZE': normalized_entries
})

scores_df['COMPOSITE_SCORE'] = scores_df.mean(axis=1)

json_data = {}
for airport_id in scores_df.index:
    json_data[str(airport_id)] = {
        'labels': ['cancelled', 'delayed', 'diverted', 'airport size', 'composite_score'],
        'data': scores_df.loc[airport_id].tolist()
    }

print("Data saved to airport_scores.json")
with open('./data/airport_scores.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)





Data saved to airport_scores.json


### Delay types

In [13]:
df = pd.read_csv('./data/flight_delays_data.csv')

df = df.fillna(0)

# filter to keep entries with delay
df = df[df['DEP_DELAY'] > 0]

delay_types_df = pd.DataFrame({
    'AIRPORT_ID': df['ORIGIN_AIRPORT_ID'],
    'CARRIER_DELAY_PCT': df['CARRIER_DELAY'] / df['DEP_DELAY'] * 100,
    'WEATHER_DELAY_PCT': df['WEATHER_DELAY'] / df['DEP_DELAY'] * 100,
    'NAS_DELAY_PCT': df['NAS_DELAY'] / df['DEP_DELAY'] * 100,
    'SECURITY_DELAY_PCT':  df['SECURITY_DELAY'] / df['DEP_DELAY'] * 100
})

delay_types_df['OTHER_DELAY_PCT'] = 100 - (
    delay_types_df['CARRIER_DELAY_PCT'] + 
    delay_types_df['WEATHER_DELAY_PCT'] + 
    delay_types_df['NAS_DELAY_PCT'] + 
    delay_types_df['SECURITY_DELAY_PCT']
)

delay_types_df = delay_types_df.groupby('AIRPORT_ID')[['CARRIER_DELAY_PCT', 'WEATHER_DELAY_PCT', 'NAS_DELAY_PCT', 'SECURITY_DELAY_PCT', 'OTHER_DELAY_PCT']].mean()

json_data = {}
for airport_id in delay_types_df.index:
    json_data[str(airport_id)] = {
        'labels': ['carier delay', 'weather delay', 'national airspace system error', 'security delay', 'other'],
        'data': delay_types_df.loc[airport_id].tolist()
    }

with open('./data/delay_types.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)



### Flight diversions

In [15]:

df_airports = pd.read_json('./data/airports.json')

df_delay = pd.read_csv('./data/flight_delays_data.csv')
df_delay = df_delay.drop(columns=['YEAR', 'MONTH', 'OP_UNIQUE_CARRIER', 
                                  'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 
                                  'DEST_AIRPORT_SEQ_ID','DEST_CITY_MARKET_ID', 'DEST_STATE_ABR', 
                                  'DEST_STATE_NM', 'DEST_CITY_NAME'])
# df_delay.head(100)

# calculate the number of divertion per airport, and sort them in descending order
df_diversion = df_delay[df_delay['DIVERTED'] == 1]
df_diversion = df_diversion.groupby('ORIGIN_AIRPORT_ID').size().reset_index(name='DIVERTED_COUNT').sort_values(by='DIVERTED_COUNT', ascending=False)

# keep only top 20 airports with the most number of divertions, drop all columns but ORIGIN_AIRPORT_ID and DIVERTED_COUNT
top_20_diverted_airports = df_diversion.head(20)[['ORIGIN_AIRPORT_ID', 'DIVERTED_COUNT']].merge(df_airports, left_on='ORIGIN_AIRPORT_ID', right_on='AIRPORT_ID', how='left').drop(columns=['ORIGIN_AIRPORT_ID'])
top_20_diverted_airports['DIVERTED_COUNT'] = top_20_diverted_airports['DIVERTED_COUNT'] * 12
print(top_20_diverted_airports)

json_data = {
    'labels': top_20_diverted_airports['AIRPORT'].tolist(),
    'data': top_20_diverted_airports['DIVERTED_COUNT'].tolist()
}

# Save to a JSON file
with open('./data/diverted_flights.json', 'w') as json_file:
    json.dump(json_data, json_file)


    DIVERTED_COUNT  AIRPORT_ID AIRPORT  \
0             1296       11298     DFW   
1             1212       14747     SEA   
2             1212       13930     ORD   
3              972       10397     ATL   
4              780       11292     DEN   
5              624       12266     IAH   
6              612       11057     CLT   
7              564       12892     LAX   
8              540       13303     MIA   
9              468       11697     FLL   
10             432       10721     BOS   
11             420       14107     PHX   
12             420       12953     LGA   
13             408       11618     EWR   
14             408       13204     MCO   
15             408       14100     PHL   
16             372       13487     MSP   
17             372       14869     SLC   
18             348       12889     LAS   
19             312       14771     SFO   

                       DISPLAY_AIRPORT_NAME DISPLAY_AIRPORT_CITY_NAME_FULL  \
0                Dallas Fort Worth Regi